In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vectorbt as vbt

In [2]:
df = pd.read_csv('../data/BTCUSDT_1h_2020-2025.csv')

In [3]:
# import a function that returns values that can help us split the data into train and test
from helper_functions.train_test_split import wfo_rolling_split_params

# Define the parameters for the rolling split
total_candles = len(df) # Total number of candles in the dataset
number_of_windows = 10 # 2020-2025 (5 years) of data divided into 10 windows 
in_sample_percentage = 0.8 # 75% of the data will be used for training

n, window_len, set_lens = wfo_rolling_split_params(total_candles=len(df),
                                                   insample_percentage=in_sample_percentage,
                                                   n=number_of_windows)

# Split data into in-sample and out-of-sample sets
(in_ohlcv, in_indexes), (out_ohlcv, out_indexes) = df.vbt.rolling_split(
    n=n,
    window_len=window_len,
    set_lens=set_lens
)

# we can also display the in-sample and out-of-sample windows to better undersand the split
# df.vbt.rolling_split(
#     n=n,
#     window_len=window_len,
#     set_lens=set_lens,
#     trace_names=['In-Sample', 'Out-of-Sample'],
#     plot=True).update_layout(width=800, height=400)


In [4]:
# parameters_ranges = {
#     'regression_window': np.arange(10, 101, 10),
#     'trend_strength': np.linspace(1.0, 2.5, 6),
#     'vol_threshold': np.linspace(1, 2, 6)
# }
# parameters_ranges

In [5]:

parameters_ranges = {
    'regression_window': np.arange(10, 61, 5),
    'bb_mult': np.linspace(1, 3, 8),
    'vol_threshold': np.linspace(1.5, 2.5, 6)
}
parameters_ranges

{'regression_window': array([10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]),
 'bb_mult': array([1.        , 1.28571429, 1.57142857, 1.85714286, 2.14285714,
        2.42857143, 2.71428571, 3.        ]),
 'vol_threshold': array([1.5, 1.7, 1.9, 2.1, 2.3, 2.5])}

In [6]:
from wfo import WFOptimizer


optimizer = WFOptimizer()

results = optimizer.optimize(
    in_ohlcv = in_ohlcv,
    out_ohlcv = out_ohlcv,
    in_indexes = in_indexes,
    out_indexes = out_indexes,
    param_ranges=parameters_ranges,
    freq='1h',
    fees=0.0005,
    slippage=0,
    init_cash=1000,
    auto_select=True,
    subfolder='results/mean_reversion',
)

2025-01-31 14:32:41,491 - INFO - Starting optimization with 8 cores
Window 1: 100%|██████████| 528/528 [00:52<00:00, 10.04it/s]
2025-01-31 14:33:45,716 - INFO - 
Window 1 Results:
2025-01-31 14:33:45,717 - INFO - In-sample top Sharpe: 1.7564
2025-01-31 14:33:45,718 - INFO - Chosen parameters: {'regression_window': np.int64(15), 'bb_mult': np.float64(2.7142857142857144), 'vol_threshold': np.float64(1.5)}
2025-01-31 14:33:45,718 - INFO - Out-of-sample Sharpe: -1.1468
Window 2: 100%|██████████| 528/528 [00:17<00:00, 30.69it/s]
2025-01-31 14:34:03,229 - INFO - 
Window 2 Results:
2025-01-31 14:34:03,230 - INFO - In-sample top Sharpe: 2.0540
2025-01-31 14:34:03,230 - INFO - Chosen parameters: {'regression_window': np.int64(10), 'bb_mult': np.float64(1.8571428571428572), 'vol_threshold': np.float64(2.3)}
2025-01-31 14:34:03,231 - INFO - Out-of-sample Sharpe: 0.7037
Window 3: 100%|██████████| 528/528 [00:16<00:00, 31.45it/s]
2025-01-31 14:34:20,325 - INFO - 
Window 3 Results:
2025-01-31 14:34:

In [7]:
# import matplotlib.pyplot as plt

# # Set up the figure and axes
# plt.figure(figsize=(14, 8))

# # Plot the Close price
# plt.plot(df.index, df['Close'], label='Close Price', color='blue', alpha=0.6)

# # Highlight Trend-Following Regime
# plt.fill_between(
#     df.index,
#     df['Close'],
#     where=(df['Regime'] == 'Trend'),
#     color='red',
#     alpha=0.3,
#     label='Trend-Following Regime'
# )

# # Highlight Mean-Reverting Regime
# plt.fill_between(
#     df.index,
#     df['Close'],
#     where=(df['Regime'] == 'Mean-Revert'),
#     color='green',
#     alpha=0.3,
#     label='Mean-Reverting Regime'
# )

# # Add labels, legend, and title
# plt.title('Market Regimes Based on Volatility Filter', fontsize=16)
# plt.xlabel('Date/Time', fontsize=14)
# plt.ylabel('Close Price', fontsize=14)
# plt.legend(loc='upper left', fontsize=12)
# plt.grid(alpha=0.3)
# plt.tight_layout()

# # Show the plot
# plt.show()
